In [1]:
import sys
sys.path.append('../')
from xd import *
from utils import *
import esutil
import healpy as hp
from systematics import *
from systematics_module import *
import os

%matplotlib inline

%load_ext autoreload
%autoreload 2

In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
import numpy as np
from run_systematics import sys_iteration, weightmultiply, fitting_allSP, calling_sysMap

In [3]:
# calling map 
path = '/fs/scratch/PCON0003/warner785/bwarner/'
LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAOSOF_22_3_v2p2.fits')
# 'Y1LSSmask_v2_redlimcut_il22_seeil4.0_4096ring.fits'
#GoldMask = callingEliGoldMask()
LSSMask = LSSGoldmask
GoldMask = LSSGoldmask

pixra, pixdec = hp.pix2ang(nside=4096,ipix=GoldMask['PIXEL'],nest=True,lonlat=True)

LSSMask = LSSMask[pixdec >-3.0 ]
#GoldMask_st82 = Cuts.SpatialCuts(GoldMask, ra=320, ra2=360, dec=-2, dec2=2)
GoldMask_st82 = GoldMask[ pixdec > -3.0 ]
GoldMask_spt = GoldMask[ pixdec < -3.0 ]
#GoldMask_spt = Cuts.SpatialCuts(GoldMask_spt, ra=0, ra2 = 100, dec=-52, dec2 = -48)

pixarea = hp.nside2pixarea( 4096, degrees = True)
sptnpix = GoldMask_spt['PIXEL'].size #hp.get_map_size( GoldMask_spt['PIXEL'] )
st82npix =  GoldMask_st82['PIXEL'].size # hp.get_map_size( GoldMask_st82 )
SPTMaparea = pixarea * sptnpix
ST82Maparea = pixarea * st82npix

In [4]:
sys.path.append('../../')
sys.path.append('/DMASS-analysis/measurements/code_py3/')
import esutil
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
os.chdir('../../DMASS-analysis/measurements/code_py3/')
print(os.getcwd())
from calling_catalogs import *
rootdir = '/users/PCON0003/warner785/DMASS-analysis/measurements/clustering/'
from ggl_mcal import run_ng_jk, run_nn_jk, run_nk_jk, compute_eix_jk, compute_Rgamma_jk, construct_jk, compute_jkcov, save_results

/users/PCON0003/warner785/DMASS-analysis/measurements/code_py3


In [5]:
def keepGoodRegion(des, hpInd = False, balrog=None):
    import healpy as hp
    import fitsio
    # 25 is the faintest object detected by DES
    # objects larger than 25 considered as Noise
    
    path = '/fs/scratch/PCON0003/warner785/bwarner/'
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v2_il22_seeil4.0_nside4096ring_redlimcut.fits')
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits')
    LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAOSOF_22_3_v2p2.fits')
    ringhp = hp.nest2ring(4096, [LSSGoldmask['PIXEL']])
    #Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits
    #frac_cut = LSSGoldmask['FRAC'] > 0.8
    #ind_good_ring = LSSGoldmask['PIXEL'][frac_cut]
    ind_good_ring = ringhp
    
    # healpixify the catalog.
    nside=4096
    # Convert silly ra/dec to silly HP angular coordinates.
    phi = des['RA'] * np.pi / 180.0
    theta = ( 90.0 - des['DEC'] ) * np.pi/180.0

    hpInd = hp.ang2pix(nside,theta,phi,nest=False)
    keep = np.in1d(hpInd, ind_good_ring)
    des = des[keep]
    if hpInd is True:
        return ind_good_ring
    else:
        return des

In [6]:
def ra_dec_to_xyz(ra, dec):
    """Convert ra & dec to Euclidean points
    Parameters
    ----------
    ra, dec : ndarrays
    Returns
    x, y, z : ndarrays
    """
    sin_ra = np.sin(ra * np.pi / 180.)
    cos_ra = np.cos(ra * np.pi / 180.)

    sin_dec = np.sin(np.pi / 2 - dec * np.pi / 180.)
    cos_dec = np.cos(np.pi / 2 - dec * np.pi / 180.)

    return (cos_ra * sin_dec,
            sin_ra * sin_dec,
            cos_dec)

def uniform_sphere(RAlim, DEClim, size=1):
    """Draw a uniform sample on a sphere
    Parameters
    ----------
    RAlim : tuple
        select Right Ascension between RAlim[0] and RAlim[1]
        units are degrees
    DEClim : tuple
        select Declination between DEClim[0] and DEClim[1]
    size : int (optional)
        the size of the random arrays to return (default = 1)
    Returns
    -------
    RA, DEC : ndarray
        the random sample on the sphere within the given limits.
        arrays have shape equal to size.
    """
    zlim = np.sin(np.pi * np.asarray(DEClim) / 180.)

    z = zlim[0] + (zlim[1] - zlim[0]) * np.random.random(size)
    DEC = (180. / np.pi) * np.arcsin(z)
    RA = RAlim[0] + (RAlim[1] - RAlim[0]) * np.random.random(size)
    
    return RA, DEC

def uniform_random_on_sphere(data, size = None ):
    ra = data['RA']
    dec = data['DEC']
    
    n_features = ra.size
    #size = 100 * data.size
    
    # draw a random sample with N points
    ra_R, dec_R = uniform_sphere((min(ra), max(ra)),
                                 (min(dec), max(dec)),
                                 size)
    #data = np.asarray(ra_dec_to_xyz(ra, dec), order='F').T
    #data_R = np.asarray(ra_dec_to_xyz(ra_R, dec_R), order='F').T
    
    #random redshift distribution
    
    data_R = np.zeros((ra_R.size,), dtype=[('RA', 'float'), ('DEC', 'float')])
    data_R['RA'] = ra_R
    data_R['DEC'] = dec_R
                              
    return data_R

## ---------------------------------------------

In [7]:
# import DMASS in validation region
#lens, randoms = calling_lens_catalog('/fs/scratch/PCON0003/warner785/bwarner/dmass_y1_public_v1.fits')
dmass_val, randoms = calling_lens_catalog('/users/PCON0003/warner785/DMASSY3/output/test/train_cat/y3/dmass_st82_DET200.fits')

random_val = uniform_random_on_sphere(dmass_val, size = 50 * dmass_val.size)
# applying LSS mask 
random_val = keepGoodRegion(random_val)

Calculating DMASS systematic weights...
Resulting catalog size
DMASS= 28441.7573735908
randoms= 5864645


In [8]:
print(dmass_val['HPIX_4096']) 

[71771941 71771626 71773248 ... 72968986 72905874 72968681]


In [9]:
plt.rcParams.update({
  "text.usetex": False,
  "font.family": "Helvetica"
})

In [10]:
random_val = appendColumn(random_val, value=np.ones(random_val.size), name='WEIGHT')
randomsind_val = np.random.choice( random_val.size, size=int(random_val.size/3.) )
random_val = random_val[randomsind_val]

In [11]:
# import DMASS in training region
#lens, randoms = calling_lens_catalog('/fs/scratch/PCON0003/warner785/bwarner/dmass_y1_public_v1.fits')
dmass_train, randoms = calling_lens_catalog('/users/PCON0003/warner785/DMASSY3/output/test/train_cat/y3/dmass_st82_DET200train.fits')

random_train = uniform_random_on_sphere(dmass_train, size = 50 * dmass_train.size)
# applying LSS mask 
random_train = keepGoodRegion(random_train)

Calculating DMASS systematic weights...
Resulting catalog size
DMASS= 20605.043776036357
randoms= 5864645


In [12]:
random_train = appendColumn(random_train, value=np.ones(random_train.size), name='WEIGHT')
randomsind_train = np.random.choice( random_train.size, size=int(random_train.size/3.) )
random_train = random_train[randomsind_train]

In [13]:
# calling CMASS

import fitsio
cmass = esutil.io.read('/fs/scratch/PCON0003/warner785/bwarner/cmass-dr12v4-S-Reid-full.dat.fits')
train_sample = esutil.io.read('/fs/scratch/PCON0003/warner785/bwarner/cmass-dr12v4-S-Reid-full.dat.fits')
#train_sample = esutil.io.read('/global/cscratch1/sd/bwarner/galaxy_DR11v1_CMASS_South-photoObj.fits.gz')
print('total num of train', train_sample.size)
print('\n--------------------------------\n applying DES veto mask to CMASS\n--------------------------------')   
train_sample = keepGoodRegion(train_sample)

#fitsio.write( output_dir+'/cmass_in_st82.fits', train_sample)

print('num of train_sample after des veto', train_sample.size)

total num of train 258884

--------------------------------
 applying DES veto mask to CMASS
--------------------------------
num of train_sample after des veto 61031


In [14]:
cmass = train_sample

rand_cmass = uniform_random_on_sphere(cmass, size = 200 * cmass.size)
rand_cmass = keepGoodRegion(rand_cmass)
rand_cmass = rand_cmass[ rand_cmass['DEC'] > -3.0 ]

In [15]:
# calling systematic maps
#sysMap = calling_sysMap( properties=properties, kind='SPT', nside=4096 )

# location of maps: /fs/scratch/PCON0003/warner785/bwarner/
# y3a2_g_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz 
# y3a2_i_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz        
# y3a2_r_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz        
# y3a2_z_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz 

input_path = '/fs/scratch/PCON0003/warner785/bwarner/'
input_keyword = 'y3a2_g' 
sysMap = io.SearchAndCallFits(path = input_path, keyword = input_keyword)


Calling catalogs >>>
y3a2_g_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz
reading 1/1 /fs/scratch/PCON0003/warner785/bwarner/y3a2_g_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz


In [16]:
print(dmass_val.dtype.names)
print()
print(sysMap.dtype.names)

('COADD_OBJECT_ID', 'RA', 'DEC', 'HPIX_4096', 'MAG_DETMODEL_G', 'MAGERR_DETMODEL_G', 'MAG_DETMODEL_R', 'MAGERR_DETMODEL_R', 'MAG_DETMODEL_I', 'MAGERR_DETMODEL_I', 'MAG_DETMODEL_Z', 'MAGERR_DETMODEL_Z', 'MOF_CM_MAG_ERR_G', 'MOF_CM_MAG_ERR_R', 'MOF_CM_MAG_ERR_I', 'MOF_CM_MAG_ERR_Z', 'MOF_CM_MAG_CORRECTED_G', 'MOF_CM_MAG_CORRECTED_R', 'MOF_CM_MAG_CORRECTED_I', 'MOF_CM_MAG_CORRECTED_Z', 'SOF_CM_MAG_ERR_G', 'SOF_CM_MAG_ERR_R', 'SOF_CM_MAG_ERR_I', 'SOF_CM_MAG_ERR_Z', 'SOF_CM_MAG_CORRECTED_G', 'SOF_CM_MAG_CORRECTED_R', 'SOF_CM_MAG_CORRECTED_I', 'SOF_CM_MAG_CORRECTED_Z', 'EXTENDED_CLASS_MASH_SOF', 'CMASS_PROB', 'WEIGHT')

('PIXEL', 'SIGNAL')


In [17]:
minimum = np.min(sysMap['PIXEL'])
maximum = np.max(sysMap['PIXEL'])
print(sysMap['SIGNAL'])
#np.min(sysMap['PIXEL'])
#np.max(sysMap['PIXEL'])
minpix = np.min(dmass_val['HPIX_4096'])
maxpix = np.max(dmass_val['HPIX_4096'])
#pbin = np.linspace(minimum, maximum, 13)
print(minpix)
print(maxpix)
#M, _ = np.histogram(dmass_val, bins=pbin) #shaded gray in plot

#number density of each
sysMaps_len = sysMap['PIXEL'].size
dmass_len = dmass_val['HPIX_4096'].size
print(sysMaps_len)

[1.0857818 1.0857818 1.0857818 ... 1.0808538 1.0808538 1.0808538]
35370
150977214
25496694


In [18]:
#compare two arrays: mask first array
# sort dmass value to be chronilogical order:
index_mask = np.argsort(dmass_val['HPIX_4096'])

#print(index_mask)
#hpix_smallest = dmass_val['HPIX_4096'][2111516]

dmass_chron = dmass_val[index_mask] # ordered by hpix values


#dim_mask = np.in1d(sysMap['PIXEL'], dmass_chron['HPIX_4096'], assume_unique=False, invert=False)
                    
#print(dim_mask)
#sys_pix = sysMap['PIXEL'][dim_mask]
#print(sys_pix)

In [19]:
print(dmass_chron['HPIX_4096'])

[35370 35370 35456 ... 150977212 150977212 150977214]


In [ ]:
# don't use this one

dmass_signal = []

for x in range(sysMaps_len):
    if sysMap['PIXEL'][x] > minpix or sysMap['PIXEL'][x] == minpix:
        if sysMap['PIXEL'][x] < maxpix or sysMap['PIXEL'][x] == maxpix:
            for y in range(dmass_len):
                if dmass_val['HPIX_4096'][y] == sysMap['PIXEL'][x]:
                    dmass_signal.append(sysMap['SIGNAL'][x])
                    
print(dmass_signal)

In [ ]:
# don't run this
# save the corresponding dmass signal
outdir = '../output/test/train_cat/y3/'
os.makedirs(outdir, exist_ok=True)
esutil.io.write( outdir+'dmass_signal.fits', dmass_signal, overwrite=True)

In [ ]:
# number of galaxies in each pixel:

minimum = np.min(sysMap['SIGNAL']) #FWHM signal
maximum = np.max(sysMap['SIGNAL'])

print(minimum)
print(maximum)
pbin = np.linspace(minimum, maximum, 13)

# dmass catalog turned into healpix map
# counting number of galaxies in each healpix pixel -- some numpy?
# dividing healpix pixel area --- corresponding area to 4096 resolution -- to get number density (plus mask -- later) FRACDET

# 1) galaxy number counts and area for each healpix pixel -- match healpix with area map?

dmass_counts = []
dmass_healpix = []
value_last = 0
arr = dmass_chron['HPIX_4096']

#check:
count = arr[arr==dmass_chron['HPIX_4096'][0]].shape[0]
print(count)

for x in range(len(dmass_chron['HPIX_4096'])):
    value = dmass_chron['HPIX_4096'][x]
    if value != value_last:
        count = arr[arr==value].shape[0]
    dmass_counts.append(count)
    dmass_healpix.append(value)

print(dmass_counts)
print(dmass_healpix)

0.7189579
1.7447658
2


In [ ]:
# save the dmass counts and healpix arrays
outdir = '../output/test/train_cat/y3/'
os.makedirs(outdir, exist_ok=True)
esutil.io.write( outdir+'dmass_counts.fits', dmass_counts, overwrite=True)

outdir = '../output/test/train_cat/y3/'
os.makedirs(outdir, exist_ok=True)
esutil.io.write( outdir+'dmass_healpix.fits', dmass_healpix, overwrite=True)

In [ ]:
# 2) match healpix map with systematic maps (using same method as above)

dim_mask = np.in1d(sysMap['PIXEL'], dmass_healpix, assume_unique=False, invert=False)
                    
#print(dim_mask)
sys_pix = sysMap['PIXEL'][dim_mask]
print(sys_pix)

In [ ]:
# 3) array with: healpix pixel value, galaxy counts, area --- binned in signal histogram
# for all signal bins : total number counts in healpix pixels / totsl area of healpix pixels used in same bin --- number density

#N, _ = np.histogram(matched_dmass_number_counts, bins=pbin)

# change to number density: divide by area? 